# Day 24
## Part 1
I'm going to use graphlib to sort the DAG.

In [4]:
from collections import namedtuple
from pyrsistent import pmap

Gate = namedtuple("Gate", "input1 op input2")

ops = {
    "AND": lambda x, y: x & y,
    "OR": lambda x, y: x | y,
    "XOR": lambda x, y: x ^ y
}

def parse_data(s):
    chunks = s.strip().split("\n\n")

    values = {}
    for line in chunks[0].splitlines():
        wire, value = line.split(": ")
        values[wire] = int(value)

    gates = {}
    for line in chunks[1].splitlines():
        inputs, output = line.split(" -> ")
        gates[output] = Gate(*inputs.split())

    return pmap(values), pmap(gates)

test_data = parse_data("""x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02""")

test_data

(pmap({'y01': 1, 'x00': 1, 'x01': 1, 'y00': 0, 'x02': 1, 'y02': 0}),
 pmap({'z00': Gate(input1='x00', op='AND', input2='y00'), 'z02': Gate(input1='x02', op='OR', input2='y02'), 'z01': Gate(input1='x01', op='XOR', input2='y01')}))

In [5]:
import graphlib

def part_1(data):
    values, gates = data

    ts = graphlib.TopologicalSorter()
    for g in gates:
        ts.add(g, gates[g].input1, gates[g].input2)

    for wire in ts.static_order():
        if wire not in values:
            g = gates[wire]
            values = values.set(
                wire, ops[g.op](values[g.input1], values[g.input2])
            )

    zs = [wire for wire in values if wire.startswith("z")]
    binary = "".join(str(values[x]) for x in reversed(sorted(zs)))
    return int(binary, 2)
    
part_1(test_data)

4

In [6]:
test_data_2 = parse_data("""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj""")

part_1(test_data_2)

2024

In [9]:
data = parse_data(open("input").read())

part_1(data)

69201640933606

## Part 2

Chunky! How many bits are in the values to be added?

In [10]:
values, gates = data
len([v for v in values if v.startswith("x")])

45

I did the [nand2tetris](https://www.nand2tetris.org/) course a few years ago, and my solution for binary addition looks like
```
CHIP HalfAdder {
    IN a, b;    // 1-bit inputs
    OUT sum,    // Right bit of a + b 
        carry;  // Left bit of a + b

    PARTS:
    Xor (a = a, b = b, out = sum);
    And (a = a, b = b, out = carry);
}

CHIP FullAdder {
    IN a, b, c;  // 1-bit inputs
    OUT sum,     // Right bit of a + b + c
        carry;   // Left bit of a + b + c

    PARTS:
    HalfAdder (a = a, b = b, sum = sumab, carry = carryab);
    HalfAdder (a = sumab, b = c, sum = sum, carry = carryabc);
    Or (a = carryab, b = carryabc, out = carry);
}

CHIP Add16 {
    IN a[16], b[16];
    OUT out[16];

    PARTS:
    HalfAdder (a = a[0], b = b[0], sum = out[0], carry = carry0);
    FullAdder (a = a[1], b = b[1], c = carry0, sum = out[1], carry = carry1);
    FullAdder (a = a[2], b = b[2], c = carry1, sum = out[2], carry = carry2);
    FullAdder (a = a[3], b = b[3], c = carry2, sum = out[3], carry = carry3);
    ...
}
```

So a three bit addition would look like
```
x0 XOR y0 -> z0
x0 AND y0 -> carry0
x1 XOR y1 -> sum1
x1 AND y1 -> carry1_1
sum1 XOR carry0 -> z1
sum1 AND carry0 -> carry0_1
carry1_1 XOR carry0_1 -> carry1
x2 XOR y2 -> sum2
x2 AND y2 -> carry2_1
sum2 XOR carry1 -> z2
sum2 AND carry1 -> carry1_2
carry2_1 XOR carry1_2 -> carry2
```

The number of gates, taking into account the last carry is not needed, would be

In [14]:
4 + 5*44 - 3

221

In [11]:
len(gates)

222

Oh, maybe an unnecessary carry is included? Or more likely I've made a mistake?

I have two ideas - generate the correct gates and compare to the ones in the data, or eyeball them in pyvis. Try the latter to start with.

In [47]:
values, gates = data

from pyvis.network import Network
g = Network(notebook=True, directed=True)
for value in values:
    g.add_node(value, color="#999999")

ts = graphlib.TopologicalSorter()
for gate in gates:
    ts.add(gate, gates[gate].input1, gates[gate].input2)

for wire in ts.static_order():
    if wire not in g.get_nodes():
        if wire in gates:
            gate = gates[wire]
            match gate.op:
                case "AND": color="#ff0000"
                case "OR": color="#00ff00"
                case "XOR": color="#0000ff"

            g.add_node(wire, label=f"{gate.op} -> {wire}", color=color)
            g.add_edge(gate.input1, wire)
            g.add_edge(gate.input2, wire)

g.show("graph.html")        

graph.html


Eyeballing it is too hard, I'll have to automate it somehow.